# DOWNLOADING DATA AND PREPARING FOR FURTHER CLEANING

-------

## 0. IMPORTS

In [1]:
import pandas as pd 
import numpy as np
import wrds
import os
import re
from utils import load_data, save_data

-------

## 1. LOADING THE DATA

### 1.1 CONNECTING TO THE WHARTON DATABASE (Skip this step if you don't want to download the data from the database)

In [2]:
# CONNECTING TO THE WHARTON DATABASE
wrds_db = wrds.Connection()

Loading library list...
Done


In [3]:
# Creates a subset list of countries in EU AND US to extract from wrds, the country code is given by JPK on their repository
countries = pd.read_excel("Country Classification.xlsx")
subset_countries = countries.query("region == 'north america' or region == 'europe'").excntry.tolist()

In [4]:
#downloading and extracting list of characteristics
chars = pd.read_excel('https://github.com/bkelly-lab/ReplicationCrisis/raw/master/GlobalFactors/Factor%20Details.xlsx')
chars_rel = chars[chars['abr_jkp'].notna()]['abr_jkp'].tolist()

In [5]:
# CREATE A STRING VERSION OF THE FACTORS FOR THE SQL QUERY
vars = ', '.join(map(str, chars_rel))

### 1.2 EXTRACTING THE DATA OR LOADING IT LOCALLY IF ALREADY SAVED

**CHANGE `GET_DATA` to True IF YOU WANT TO RE-FETCH THE DATA FROM THE DATABASE**

In [6]:
GET_DATA = False

In [7]:
# Starting Date for our purposes
date = '1995-01-01'

In [8]:
# Extracting the factors
if GET_DATA:
        query = f"""
                SELECT id, eom, excntry, gvkey, permno, size_grp, me, {vars}
                        FROM contrib.global_factor
                        WHERE common=1 
                        and 
                        exch_main=1 
                        and 
                        primary_sec=1 
                        and 
                        obs_main=1
                        and
                        excntry = 'USA'
                        and
                        date > '{date}'
                """

        data = wrds_db.raw_sql(query)
        save_data(data)
else:
        df = load_data()

In [10]:
# GETTING THE MONTHLY RETURN DATA AND THE INDUSTRY SECTOR
if GET_DATA:
        # Extracting the return
        query_ret = f"""
                SELECT id, eom, excntry, gvkey, permno, ret, ff49
                        FROM contrib.global_factor
                        WHERE common=1 
                        and 
                        exch_main=1 
                        and 
                        primary_sec=1 
                        and 
                        obs_main=1
                        and
                        excntry = 'USA'
                        and
                        date > '{date}'
                """

        ret = wrds_db.raw_sql(query_ret) # get data
        ret.eom = pd.to_datetime(ret.eom)
        ret.to_csv("return_GFD.csv", index = False) # save data
else:
        ret = pd.read_csv("return_GFD.csv")
        ret.eom = pd.to_datetime(ret.eom)

-------

## 2. CLEANING AND MERGING

In [11]:
# REMOVE SMALL COMPANIES
l_remove = ["micro","nano","NAN","small"]
df.size_grp = df.size_grp.fillna("NAN")
df = df[~df.size_grp.isin(l_remove)].reset_index(drop = True)

In [12]:
# REMOVING OBSERVATION IN 2024
df.eom = pd.to_datetime(df.eom)
df = df[df.eom.dt.year <= 2023].loc[:,:].reset_index(drop = True)
n_companies = len(df.id.unique())
n_dates = len(df.eom.unique())
pd.DataFrame({"Unique companies" : [n_companies], "Unique periods" : [n_dates]}).T.rename(columns = {0: "TOTAL"})

,TOTAL
Unique companies,5645
Unique periods,348


This is right as the number of periods from $k = 1995$  and  $n = 2023$  for  S = 12$ time periods per year, is: $T = (n - k + 1) * S = 348$  
  
Note that we remove the three observed months from 2024 for simplicity.

In [13]:
# RUN A CHECK
complete_data = pd.DataFrame(df.groupby("id").size()).sort_values(0,ascending = False).rename(columns = {0:"num"}).query("num == 348").shape[0]
print(f"Unfortunately only {complete_data} companies have data for every period")

Unfortunately only 226 companies have data for every period


In [14]:
# MERGE THE TARGET (Return at time t) WITH THE FACTORS
df_returns = pd.merge(df,ret[["id","eom","ret","ff49"]], how = "left", on = ["id","eom"])

### Process Overview

1. **Sort DataFrame**: The `df_returns` DataFrame is sorted by `id` and `eom` (end of month/date). So that we can look at each company's time series sequentially
2. **Loop through Rows**: For each row (except the last):
- If we reach the last observation (likely 2023) for a company, the target variable takes value `None`, as we don't have the following return
- If the return in the next row is reported more than 31 days after the current return, we set the target to `None` as again, we do not have the following return


The result is a list `y` that stores the lead of `ret` values based on the conditions above.  
After removing the last row from our dataset, we include `y` as a column.  

Example

| **id (company)**  | **eom (date)**       | **ret (time t)**  | **y (time t + 1)**|
|:-----|:----------|-----:|---------:|
| A    | 2024-01-31| 0.10 |      0.2 |
| A    | 2024-03-31| 0.20 |      NaN |
| A    | 2024-04-30| 0.15 |      NaN |
| B    | 2024-01-31| 0.05 |      0.10 |
| B    | 2024-02-28| 0.10 |      NaN |
| C    | 2024-01-31| 0.25 |      0.30 |
| C    | 2024-02-28| 0.30 |      0.35 |
| C    | 2024-03-31| 0.35 |      NaN |
| D    | 2024-03-05| 0.40 |      NaN |

In [15]:
# Sort
df_returns.sort_values(["id","eom"], inplace = True)
df_returns.reset_index(drop = True, inplace = True)

# Loop through rows and store the value in y
y = []
for i in range(len(df_returns) - 1):
    current_id = df_returns.loc[i,"id"]
    current_date = df_returns.loc[i,"eom"]
    next_id = df_returns.loc[i + 1,"id"]
    next_date = df_returns.loc[i + 1,"eom"]

    # If we the next row contains another company's data append None and move on
    if current_id != next_id:
        y.append(None)
        continue
    
    # If the next row contains data for the same company less than a month apart append the return at t+1
    if (next_date - current_date).days <= 31:
        r = df_returns.loc[i + 1,"ret"]
        y.append(r)
    # If the next row contains data more than one month apart, append none
    else:
        y.append(None)
        

In [16]:
# Checks
print(f"There are a total of {y.count(None)} observations lost\n\nOf these: {y.count(None) - len(df_returns.id.unique())} are lost because the return of the following month is missing. \nThe remaining {len(df_returns.id.unique())} are lost because the last observation for each company is not followed by any return, hence we lose one observation per company.")

There are a total of 13907 observations lost

Of these: 8262 are lost because the return of the following month is missing. 
The remaining 5645 are lost because the last observation for each company is not followed by any return, hence we lose one observation per company.


In [17]:
# Store the target variable in the datagframe
df_returns = df_returns.iloc[:-1]
df_returns.loc[:,"y"] = y
df_final = df_returns[~df_returns.y.isna()].reset_index(drop = True)

In [ ]:
# Save the dataframe
SAVE = False
if SAVE:
    df_final.to_csv("GFD_final.csv", index = False)